In [1]:
# @title Install dependencies
%%capture
!pip install plotly dash

In [2]:
# @title Plot the GBM1000 UMAP
coloring = "Natural compound-likeness" # @param ["Natural compound-likeness","ChEMBL bioactivity","Gbma selectivity","Gbma lethality"]
show_grid = False # @param {"type":"boolean","placeholder":"False"}

import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, no_update, callback


df = pd.read_csv('https://raw.githubusercontent.com/OlivierBeq/GBM1000_UMAP/refs/heads/master/data/UMAP_data.tsv.xz', sep='\t')
df["tgt_pchembl_value"] = df["tgt_pchembl_value"].fillna(-1)
df["delta_healthy_Gbma"] = df["delta_healthy_Gbma"].fillna(-1)
df["zscore_Gbma"] = df["zscore_Gbma"].fillna(100)

if coloring == "Natural compound-likeness":
    color = df["sp3_fraction"]
    colorscale = 'viridis'
    title = 'Natural<br>compound-likeness'
    legend = 'Fraction Csp3'
    cmin, cmax = 0, 1
elif coloring == "ChEMBL bioactivity":
    color = df["tgt_pchembl_value"]
    colorscale = ["white", "red"]
    title = 'ChEMBL bioactivity<br>(negative log-scale)'
    legend = 'Bioactivity'
    cmin, cmax = 6, 10
elif coloring == "Gbma selectivity":
    color = df["delta_healthy_Gbma"]
    colorscale = ["white", "red"]
    title = 'Gbma selectivity'
    legend = 'Selectivity'
    cmin, cmax = -0.5, 0.5
elif coloring == "Gbma lethality":
    color = df["zscore_Gbma"]
    colorscale = ["red", "white"]
    title = 'Gbma lethality'
    legend = 'Lethality'
    cmin, cmax = -3, 0

fig = go.Figure(data=[
    go.Scatter(
        x=df["UMAP1"],
        y=df["UMAP2"],
        mode="markers",
        marker=dict(
            colorscale=colorscale,
            color=color,
            colorbar={"title": title},
            line={"color": "#444",
                  'width': (0.5
                            if coloring != "Natural compound-likeness"
                            else 0)},
            opacity=0.6,
            cmin=cmin,
            cmax=cmax,
        )
    )
])
fig.update_traces(hoverinfo="none", hovertemplate=None)

fig.update_layout(
    width=825,
    height=600,
    xaxis=dict(title='UMAP1'),
    yaxis=dict(title='UMAP2'),
    plot_bgcolor='rgba(255,255,255,0.1)'
)
if show_grid:
    fig.update_xaxes(showgrid=True, gridcolor='grey', griddash="dot")
    fig.update_yaxes(showgrid=True, gridcolor='grey', griddash="dot",
                    zeroline=True, zerolinewidth=1, zerolinecolor='grey')

# Set up the app now
app = Dash()

app.layout = html.Div(
    className="container",
    children=[
        dcc.Graph(id="graph-2-dcc", figure=fig, clear_on_unhover=True),
        dcc.Tooltip(id="graph-tooltip-2", direction='bottom'),
    ],
)

@callback(
    Output("graph-tooltip-2", "show"),
    Output("graph-tooltip-2", "bbox"),
    Output("graph-tooltip-2", "children"),
    Output("graph-tooltip-2", "direction"),

    Input("graph-2-dcc", "hoverData"),
)
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update, no_update

    point_x, point_y = hoverData['points'][0]['x'], hoverData['points'][0]['y']
    row = df.query('(UMAP1 == @point_x) and (UMAP2 == @point_y)')
    id, name = row['GBMdbID'].item(), row['Name'].item()
    if coloring == "Natural compound-likeness":
        legend_text = row['sp3_fraction'].item()
    elif coloring == "ChEMBL bioactivity":
        legend_text = row['tgt_pchembl_value'].item()
        legend_text = "N/A" if legend_text == -1 else f'{legend_text:.2f}'
    elif coloring == "Gbma selectivity":
        legend_text = row['delta_healthy_Gbma'].item()
        legend_text = "N/A" if legend_text == -1 else f'{legend_text:.2f}'
    elif coloring == "Gbma lethality":
        legend_text = row['zscore_Gbma'].item()
        legend_text = "N/A" if legend_text == 100 else f'{legend_text:.2f}'
    image_base64 = row['image_base64'].item()
    im_url = "data:image/jpeg;base64, " + image_base64

    # demo only shows the first point, but other points may also be available
    hover_data = hoverData["points"][0]
    bbox = hover_data["bbox"]

    # control the position of the tooltip
    y = hover_data["y"]
    direction = "bottom" if y > 1.5 else "top"

    children = [
        html.Div([
            html.Img(
                src=im_url,
                style={"width": "200px"},
            ),
            html.P(f"GBMdb ID: {id}", style={'margin': 0}),
            html.P(f"Name: {name}", style={'margin': 0}),
            html.P(f"{legend}: {legend_text}", style={'margin': 0}),
            ],
        style={'white-space': 'normal'}),
    ]

    return True, bbox, children, direction

if __name__ == "__main__":
    app.run(debug=True)

<IPython.core.display.Javascript object>